# <span style="color:#006E7F">_ CIBIG Metagenomic water projet analysis__ <a class="anchor"></span>  


Created by F. Sorgho (CRUN) and E. Badoum (GRAS) - Novembre 2024


# <span style="color:#006E7F">BASECALLING and QC__ <a class="anchor" id="data"></span>  
    
## <span style="color: #4CACBC;"> Creating the folder, downloading data and so on</span>  


### <span style="color: #4CACBD;"> 1. Data </span>

We will analyse a Freshwater Samples (Urban et al., 2021; https://elifesciences.org/articles/61504).  

In [ ]:
cd ~/projet_BS
mkdir -p Data
cd Data
# download your compressed Freshwater Sample, https://www.ebi.ac.uk/ena/browser/view/PRJEB34900 ERR3806859_1-ERR3806892_1
wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR380/009/ERR3806859/ERR3806859_1.fastq.gz
wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR380/006/ERR3806876/ERR3806876_1.fastq.gz
wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR380/000/ERR3806860/ERR3806860_1.fastq.gz

## <span style="color: #4CACBC;"> 2. Basecalling </span>  

Electrical signals are stocked on fast5 format files when DNA molecules are sequenced.

These signals need to be converted on standard fastq files to post-analysis.

Several training dataset models are usually used to convert fast5 to fastq. 
For this Job we used Guppy  

## <span style="color: #4CACBC;"> 3. Quality Control on Long Reads with Nanoplot </span>  

Control reads quality using Nanoplot. You can parameter this tool using --help.

In [ ]:
NanoPlot --help

In [ ]:
Install Nanoplot/1.44.0
NanoPlot -t 8 --fastq ~/projet_BS/Data/*fastq.gz --outdir NANOPLOT_files

In [ ]:
#test for one sample
NanoPlot --fastq /home/faiza/Downloads/projet_turore/database_initial/ERR3806875_1.fastq.gz --outdir ./output 

In [ ]:
# for all of sample
NanoPlot --fastq /home/faiza/Downloads/projet_turore/database_initial/ERR38068*.fastq.gz --outdir ./output_combined

Observe Stats

## <span style="color: #4CACBC;"> Use KRAKEN2 for taxonomic assignation<a class="anchor" id="kraken2"> </span>


### <span style="color: #4CACBC;"> 3.1. Download a bacterial database<a class="anchor" id="viraldb"> </span>

In [ ]:
Kraken2 --help

In [ ]:
kraken2-build --special "silva" --db kraken_database/.


In [ ]:
# Inspect the database content
kraken2-inspect --db kraken_database | head -15

### <span style="color: #4CACBC;"> 3.2. run Kraken2 <a class="anchor" id="kraken"> </span>

In [ ]:
kraken2 --db kraken_database/ /projects/medium/CIBIG_metagenomic_eaux/RAW_DATA/FASTQ_DIR/ERR38068*.fastq.gz --report report.txt --report-minimizer-data --> output_kraken

### <span style="color: #4CACBC;"> 3.3. Vizualise kraken2 output with krona<a class="anchor" id="krakenkrona"> </span>

In [ ]:
ktImportTaxonomy -m 3 -t 5 report.txt -o kraken.html 2> krakenkrona.err

## <span style="color: #4CACBC;"> 4. Use Diamond for taxonomic assignation<a class="anchor" id="diamond"> </span>

### <span style="color: #4CACBC;"> 4.1. Download bacteria bank<a class="anchor" id="bacteriadbdiamond"> </span>